In [2]:
import copy
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import datetime
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from functools import partial
from dateutil.relativedelta import relativedelta
from datetime import date
from sklearn.preprocessing import OneHotEncoder

# show all the columns when printing
pd.set_option('display.max_columns', 999)

ANCHOR_DATE = date(2001,9,30) - relativedelta(days=15248)

def convert_int_date_to_real_date(int_date):
    return ANCHOR_DATE + relativedelta(days=int_date)

def convert_real_date_to_int_date(real_date):
    return (real_date-ANCHOR_DATE).days

# Test Date Crap
fy_end = date(2018,9,30)
print(convert_int_date_to_real_date(15248))
print(convert_real_date_to_int_date(fy_end))
print(convert_real_date_to_int_date(fy_end)-convert_real_date_to_int_date(fy_end-relativedelta(days=350)))

2001-09-30
21457
350


In [3]:
services_cols = ["SpecEdSv",
"ILNAsv",
"AcSuppSv",
"PSEdSuppSv",
"CareerSv",
"EmplyTrSv",
"BudgetSv",
"HousEdSv",
"HlthEdSv",
"FamSuppSv",
"MentorSv",
"SILsv",
"RmBrdFASv",
"EducFinaSv",
"OthrFinaSv"]

# No special ed for prediction, very distinct from the other services theoretically
all_service_cols = [x for x in services_cols if x !="SpecEdSv"]

potential_outcomes = all_service_cols+["AllServices_Binary",
#"AllServices_Full",
"FinancialServices_Binary",
#"FinancialServices_Full",
"YoungerServices_Binary"]#,
#"YoungerServices_Full"]

# Load Data

In [4]:
YEAR = "2018"

fy_start = date(int(YEAR)-1, 10,1)
fy_end = date(int(YEAR),9,30)
fy_end_date_int = (fy_end -ANCHOR_DATE).days
fy_start_date_int = (fy_start-ANCHOR_DATE).days

## Load Services data

In [5]:
import os
services_data = pd.read_sas("/data/afcars/nytd/services/Data/SAS/services2018_f.sas7bdat")
#treat 77s and nulls as clerical errors / incomplete data
services_data = services_data[~(services_data[all_service_cols].isnull()).any(axis=1)]
services_data[~(services_data[all_service_cols] == 77).any(axis=1)]
services_data = services_data[['StFCID','FY'] + services_cols]

service_counts = services_data.groupby(['FY','StFCID']).sum().reset_index()
service_counts["AllServices_Binary"] = (service_counts[all_service_cols] > 0).sum(axis=1)

s_cols = ["SILsv","RmBrdFASv","EducFinaSv","OthrFinaSv"]
service_counts["FinancialServices_Binary"] = (service_counts[s_cols] > 0).sum(axis=1)

s_cols = ["AcSuppSv", "PSEdSuppSv", "CareerSv", "EmplyTrSv", "BudgetSv", 
          "HousEdSv", "HlthEdSv", "FamSuppSv", "MentorSv"]
service_counts["YoungerServices_Binary"] = (service_counts[s_cols] > 0).sum(axis=1)

service_counts.to_csv("services_counts.csv",index=False)

## Load AFCARS

In [6]:
afcars_prevyear = None
afcars_curryear = None
if YEAR == "2018":
    afcars_prevyear = pd.read_sas('/data/afcars/all/AFCARS Foster Care/2017/Data/SAS Files/fc17v2f.sas7bdat')
    afcars_curryear = pd.read_sas('/data/afcars/all/AFCARS Foster Care/2018/DATA/SAS Files/fc2018v1.sas7bdat') 


/opt/anaconda/lib/python3.7/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")
/opt/anaconda/lib/python3.7/site-packages/pandas/io/sas/sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]


In [7]:
afcars_prevyear.shape, afcars_curryear.shape

((691188, 105), (687402, 105))

In [8]:
service_counts['StFCID'] = service_counts.StFCID.astype("string")
afcars_curryear['StFCID'] = afcars_curryear.StFCID.astype("string")
afcars_merged_with_services = pd.merge(afcars_curryear,
                                       service_counts[service_counts.FY == int(YEAR)],
                                       on=["StFCID"],
                                       how="left")

In [9]:
afcars_merged_with_services['in_afcars_prevyear'] = afcars_merged_with_services.StFCID.isin(afcars_prevyear.StFCID.astype("string"))

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.


In [10]:
df = afcars_merged_with_services.copy()

In [11]:
df[potential_outcomes] = df[potential_outcomes].fillna(0)

In [12]:
# Tack on previous year services
prev_year_services = service_counts[service_counts.FY == (int(YEAR)-1)].copy().drop(columns=['FY',"SpecEdSv"])
prev_year_services.columns = ['StFCID'] + ['prev_'+c for c in potential_outcomes]
df = df.merge(prev_year_services, on='StFCID',how='left')
cols_to_fill = [x for x in prev_year_services.columns if x !="StFCID"]
df[cols_to_fill] = df[cols_to_fill].fillna(0)

### Create Date Columns

In [13]:
# By inclusion criterion, have all of these
df['first_removal_date_int']  = fy_end_date_int - df.Rem1Dt
df['latest_removal_date_int'] = fy_end_date_int - df.LatRemDt
df['latest_setting_date_int']  = fy_end_date_int - df.CurSetDt
df['age_at_end_int'] = fy_end_date_int - df.DOB

# These might be null
df['previous_removal_discharge_date_int']  = fy_end_date_int - df.DLstFCDt
df.loc[df.DLstFCDt.isnull(),'previous_removal_discharge_date_int'] = 0

df['latest_removal_discharge_date_int']  = fy_end_date_int - df.DoDFCDt
df.loc[df.DoDFCDt.isnull(),'latest_removal_discharge_date_int'] = 0

df['periodic_review_date_int']  = fy_end_date_int - df.PedRevDt
df.loc[df.PedRevDt.isnull(),'periodic_review_date_int'] = 0

In [14]:
# oddities of negative values, seems reasonable though that it's updated to post-reporting period?
print(df[df.first_removal_date_int < 0].shape)
print(df[df.latest_removal_date_int < 0].shape)
print(df[df.latest_setting_date_int < 0].shape)
print(df[df.previous_removal_discharge_date_int < 0].shape)
print(df[df.latest_removal_discharge_date_int < 0].shape)
print(df[df.periodic_review_date_int < 0].shape)
print(df[df.age_at_end_int < 0].shape)

(1801, 149)
(2287, 149)
(8261, 149)
(243, 149)
(3496, 149)
(16545, 149)
(2, 149)


In [15]:
df.AgeAtStart.value_counts().head()

1.0    55408
0.0    54577
2.0    49690
3.0    43994
4.0    39562
Name: AgeAtStart, dtype: int64

### Drop rows we don't want to study

In [16]:
def drop_rows(df,inclusion_criterion):
    pre = len(df)
    df = df[inclusion_criterion]
    print("N removed: {}, % Removed: {}".format(pre-len(df), (pre-len(df))/pre))
    return df

In [17]:
initial_data_length = len(df)

In [18]:
# Drop people who are older than 22 (few)
df = drop_rows(df, (df.AgeAtStart < 22) & (df.age_at_end_int <= 23*365.25))

# Drop people who are younger than 14 (limited services)
df = drop_rows(df, (df.AgeAtStart > 13) & (df.age_at_end_int >= 14*365.25))


# Drop people who were in case for > 6 months of 2018
# These variables don't always agree (errors?), so just checking all of them
df = drop_rows(df, (df.latest_removal_discharge_date_int <= 180) & 
                   (df.LatRemLOS >= 180) &
                   (df.latest_removal_date_int >= 180))

N removed: 30, % Removed: 4.3642584688435587e-05
N removed: 552641, % Removed: 0.8039911430782749
N removed: 49189, % Removed: 0.3650904394682738


In [19]:
# Drop individuals with first removal date 22+ years ago (couldn't be born)
df = drop_rows(df, df.first_removal_date_int <= (fy_start - (fy_start- relativedelta(years=22))).days)

N removed: 13, % Removed: 0.0001519721306492717


In [20]:
# Drop individuals we don't have latest removal date
# and/or first removal date 
# and/or latest setting change date for. 
df = drop_rows(df, (df.LatRemDt != 0) & (df.Rem1Dt != 0) & (df.CurSetDt) != 0)

N removed: 85, % Removed: 0.0009938149633457658


In [21]:
# Drop Youth whose Sex is unknown, we consider this a parameter of interest
df = drop_rows(df, df.Sex != 0)

N removed: 0, % Removed: 0.0


In [22]:
# Drop Youth whose Race/Ethnicity is unknown, we consider this a parameter of interest
df = drop_rows(df, df.RaceEthn != 99)

N removed: 883, % Removed: 0.010334254014325172


In [25]:
# Drop states with no service data at all
sm = df.groupby("St").AllServices_Binary.mean()
df = drop_rows(df, ~df.St.isin(sm[sm == 0].index.values.tolist()))


N removed: 2172, % Removed: 0.025685599744563095


In [26]:
# Drop people who don't have a setting LOS, seems important
df = drop_rows(df, ~df.SettingLOS.isnull())

N removed: 931, % Removed: 0.011300052191433322


In [27]:
# Drop rows with PrtsDied is null. These seemed to have a lot of nulls, too many to safely impute
df = drop_rows(df, ~df.PrtsDied.isnull())

N removed: 710, % Removed: 0.008716148199071915


In [28]:
"Total removed: {}, total percent removed: {}".format(initial_data_length - len(df),
                                                     (initial_data_length-len(df))/initial_data_length)

'Total removed: 606654, total percent removed: 0.8825316190526068'

In [29]:
"Total youth remaining: {}".format(len(df))

'Total youth remaining: 80748'

# Feature construction

In [30]:
def get_age(x):
    if x < 19:
        return str(x).replace(".0","")
    return "19+"
# Convert age to discrete, no reason to expect continuous is useful
df['age'] = df.AgeAtStart.apply(get_age).astype("string")
# Clean up state
df['St'] = df.St.astype("string")
df['St'] = df.St.str.replace("b'","").str.replace("'","")
# Create var for age/state/year
df['age_state_year'] = df.St+ "_" + df.age + "_" + YEAR
# Make Sex Binary
df['Sex'] = df.Sex -1

In [31]:
# Create variables for binary checks


df['dad_rights_term'] = ~df.TPRDadDt.isnull() * 1
df['mom_rights_term'] = ~df.TPRMomDt.isnull() * 1
df['had_periodic_review'] = df.periodic_review_date_int.apply(lambda x: 0 if x == 0 else 1)
df['was_discharged'] = df.latest_removal_discharge_date_int.apply(lambda x: 0 if x == 0 else 1)
df['previous_was_discharged'] = df.previous_removal_discharge_date_int.apply(lambda x: 0 if x == 0 else 1)


In [32]:
df['RU13_cont'] = df['RU13'].copy()

### Define all features, add log of continuous features

In [33]:


continuous_features = [ 
    "SettingLOS",
    "TotalRem",
    "NumPlep",
    'first_removal_date_int',
    'latest_removal_date_int',
    'latest_setting_date_int',
    'previous_removal_discharge_date_int',
    'latest_removal_discharge_date_int',
    'periodic_review_date_int',
    'age_at_end_int',
    "RU13"
] 

for f in continuous_features:
    if f =="RU13":
        continue
    if(df[f].min() < 0):
        df["log_"+f]=np.log(df[f]+1-df[f].min())
    else:
        df["log_"+f]=np.log(df[f]+1)
log_continuous_features = ["log_"+f for f in continuous_features if f !="RU13"]

previous_year_service_features = ["prev_"+x for x in potential_outcomes]

binary_features = [
    "had_periodic_review",
    "was_discharged",
    "previous_was_discharged",
    "dad_rights_term", 
    "mom_rights_term","AgedOut",
    "SexAbuse", "PhyAbuse", "Neglect", 
    "AAParent", "DAParent", "AAChild", "DAChild", "ChilDis", 
    "ChBehPrb", "PrtsDied", "PrtsJail", "NoCope", "Abandmnt",
    "Relinqsh", "Housing", "IsTPR", "IVEFC", "IVEAA",
    "IVAAFDC", "IVDCHSUP", "XIXMEDCD", "SSIOther", "NOA",
    "MR", "VisHear", "PhyDis", "EmotDist", 
    "OtherMed",
    "Sex",
    "in_afcars_prevyear"
]

discrete_features = [
    #"RU13",
    "RaceEthn", "CaseGoal", "CurPlSet",
    "ClinDis",  "ManRem", 
     "CtkFamSt",   "PlaceOut","St",
    "age",
    "AgeAdopt","FosFamSt","DISREASN","IsWaiting"
]


manual_features = [
    "NumPlep",
    'log_first_removal_date_int',
    'log_latest_removal_date_int',
    'log_latest_setting_date_int',
    'log_previous_removal_discharge_date_int',
    'log_latest_removal_discharge_date_int',
    "had_periodic_review",
    "was_discharged",
    "AgedOut",
    "in_afcars_prevyear",
    "RU13",
    "RaceEthn", "CaseGoal", "CurPlSet","St","age"
]



all_vars = continuous_features + log_continuous_features + previous_year_service_features + binary_features+discrete_features + potential_outcomes

# EDA

In [34]:
# Our package
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

In [35]:
vis_df = df.copy()
for var_name in discrete_features + binary_features:
    vis_df[var_name] = vis_df[var_name].astype("category")

In [ ]:
# Generate the Profiling Report
profile = ProfileReport(
    vis_df[all_vars], title="Services Pred: "+ YEAR, 
    html={"style": {"full_width": True}}, 
    sort=None,
    minimal=True
)

In [ ]:
# The Notebook Widgets Interface
profile.to_file("output.html")

# "Imputation"

EDA shows some missing values. For now, we will impute basic values

In [36]:
base_values = {"AgeAdopt" : 5, "CtkFamSt" : 5, "ClinDis": 3, "TotalRem" : 1, "NumPlep": 1,
              "latest_removal_date_int" : 180}
for col in continuous_features+binary_features+discrete_features+log_continuous_features:
    null_count  = df[col].isnull().sum()
    if null_count > 0:
        print("{} has {} nulls, replacing with {}s".format(col,null_count,base_values.get(col,0)))
    df[col] = df[col].fillna(base_values.get(col,0))

TotalRem has 42 nulls, replacing with 1s
NumPlep has 128 nulls, replacing with 1s
RU13 has 3 nulls, replacing with 0s
IVDCHSUP has 2 nulls, replacing with 0s
XIXMEDCD has 27 nulls, replacing with 0s
SSIOther has 21 nulls, replacing with 0s
MR has 1494 nulls, replacing with 0s
VisHear has 1500 nulls, replacing with 0s
PhyDis has 1493 nulls, replacing with 0s
EmotDist has 1493 nulls, replacing with 0s
OtherMed has 1493 nulls, replacing with 0s
Sex has 4 nulls, replacing with 0s
ClinDis has 2381 nulls, replacing with 3s
ManRem has 24 nulls, replacing with 0s
CtkFamSt has 1952 nulls, replacing with 5s
PlaceOut has 150 nulls, replacing with 0s
AgeAdopt has 4954 nulls, replacing with 5s
FosFamSt has 5717 nulls, replacing with 0s
log_TotalRem has 42 nulls, replacing with 0s
log_NumPlep has 128 nulls, replacing with 0s


# Prediction

### Model Setup

In [37]:
no_regression_features = {"prev_AllServices_Binary",
                         "prev_YoungerServices_Binary",
                         "prev_FinancialServices_Binary"}
def create_r_formula_string(features,outcome, factor_phrase):
    base_reg_form = "+".join([factor_phrase+"("+x+")" for x in features if (x in discrete_features and
                                                                           x != "St" and
                                                                           x != "age")])
    if factor_phrase != "i":
        if 'St' in features:
            base_reg_form += " + factor(St)"
        if 'age' in features:
            base_reg_form += " + factor(age)"
            
    base_reg_form += "+" + "+".join([x for x in features if x not in discrete_features
                                     and x not in no_regression_features])
    base_reg_form = outcome + " ~ " + base_reg_form
    if factor_phrase == "i":
        base_reg_form += " | age^St"
    return base_reg_form

r_zeroinfl_regression_call = ro.r('''
function(reg_form,train_index,test_index){
    library(glmmTMB)
    fit_zipoisson <- glmmTMB(formula(reg_form),
                        data=regression_data[train_index+1,],
                        ziformula=~St+age+factor(CaseGoal)*had_periodic_review+factor(CurPlSet)+
                        factor(DISREASN)+was_discharged,
                        family=nbinom2)
    return(predict(fit_zipoisson,newdata=regression_data[test_index+1,], type="response",na.rm=F))
}
    
''')

r_negbin_regression_call = ro.r('''
function(reg_form,train_index,test_index){
    library(fixest)
    f <- femlm(formula(reg_form),
       family="negbin",
        data=regression_data[train_index+1,],
        se="cluster",
        combine.quick=FALSE)
    return(predict(f,newdata=regression_data[test_index+1,], na.rm=F))
}
    
''')

In [38]:
import xgboost as xgb

def basic_groupby_model(groupby_vars,
                        data,
                        outcome_var,
                        train_index,
                        test_index):
    predvar = 'pred_base'+"_".join(groupby_vars)
    pred = data.iloc[train_index].groupby(groupby_vars)[outcome_var].mean().reset_index()
    pred.columns = groupby_vars+ [predvar]
    return pd.merge(data.iloc[test_index],pred,on=groupby_vars, how="left")[predvar]

rf_regressor = RandomForestRegressor(n_estimators=1000,
                                      max_features='auto',
                                      max_depth=15,
                                      min_samples_split=12,
                                      min_samples_leaf=5,
                                      n_jobs = 8,
                                      bootstrap=True, 
                                      random_state=0)
linReg = LinearRegression()

xgb_model = xgb.XGBRegressor(colsample_bytree= 0.7,
 gamma= 0.0,
 learning_rate= 0.15,
 max_depth= 6,
 min_child_weight= 3)

In [39]:
print("NA ROWS: {}".format(df[all_vars].isnull().any(axis=1).sum()))
regression_data = df[all_vars]
# Write out for any additional EDA
regression_data.to_csv(YEAR+"_final.csv", index=False)

NA ROWS: 0


In [40]:
### Send over final data to R

with localconverter(ro.default_converter + pandas2ri.converter):
    ro.globalenv['regression_data'] = ro.conversion.py2rpy(regression_data)

## Run!

In [ ]:
outcome_vars = ['AllServices_Binary',"YoungerServices_Binary", "FinancialServices_Binary"]

feature_sets = {
    "all_features" : discrete_features+log_continuous_features+binary_features+previous_year_service_features+continuous_features,
    "no_previous_year": discrete_features+log_continuous_features+binary_features+continuous_features,
}

# Create folds
kf = KFold(n_splits=10,shuffle=True,random_state=30)



result_dfs = []
fold = 0
for train_index, test_index in kf.split(regression_data):

    fold += 1
    print(fold)
    
    for outcome_var in outcome_vars:
        fold_results = {}
        
        print("\t"+outcome_var)
        
        y_train = regression_data[outcome_var].iloc[train_index]
        y_test = regression_data[outcome_var].iloc[test_index]
        
        fold_results["ind"] = test_index
        fold_results["fold"] = [fold]*len(test_index)
        fold_results["outcome"] = [outcome_var]*len(test_index)
        fold_results['pred_constant'] = [y_train.mean()] * len(y_test)
        
        # set up basic reg models (just duplicate across feature sets for simplicity)
        basic_model_partial = partial(basic_groupby_model, 
                                      data = regression_data,
                                      outcome_var=outcome_var,
                                      train_index=train_index,
                                      test_index=test_index)
        
        fold_results['pred_age'] = basic_model_partial(['age'])
        fold_results['pred_age_state'] = basic_model_partial(['age','St'])
        fold_results['pred_age_race'] = basic_model_partial(['age','RaceEthn'])
        fold_results['pred_age_race_ru13'] = basic_model_partial(['age','RaceEthn','RU13'])
        
        fold_results['pred_previous_year'] = regression_data.iloc[test_index]['prev_'+outcome_var].values.tolist()
        
        for feature_set_name, feature_set in feature_sets.items():
            
            print("\t\t"+ feature_set_name)
            
            onehotencoded_vars = pd.concat([pd.get_dummies(regression_data[var_name],
                                                           prefix=var_name)    
                                             for var_name in feature_set if var_name in discrete_features],
                                           axis=1
                                          )
            X_matrix = pd.concat([onehotencoded_vars, 
                                 regression_data[[x for x in feature_set if x not in discrete_features]]],
                                axis=1) 
            
            X_train = X_matrix.iloc[train_index]
            X_test = X_matrix.iloc[test_index]
            reg_call = create_r_formula_string(features=feature_set,
                        outcome=outcome_var,
                        factor_phrase="i")
            
            print("xgb")
            fold_results['pred_xgb_'+feature_set_name] = xgb_model.fit(X_train,y_train).predict(X_test)
            
            print("negbin")
            fold_results['pred_negbin_'+feature_set_name] = r_negbin_regression_call(reg_call,
                                                      ro.IntVector(train_index), 
                                                      ro.IntVector(test_index))

            fold_results['pred_linreg_'+feature_set_name] = linReg.fit(X_train,y_train).predict(X_test)
        fold_df = pd.DataFrame(fold_results)
        fold_df.to_csv("fold_{}.csv".format(fold))
        result_dfs.append(fold_df)


In [ ]:
# Merge predictions
results_data = pd.concat(result_dfs,axis=0)
regression_data = regression_data.assign(ind= range(len(regression_data)))

merged_with_predictions = pd.merge(regression_data,results_data,on='ind')
len(merged_with_predictions)

len(results_data), len(regression_data)

merged_with_predictions.to_csv("preds_2018_g.csv",index=False)

# Model interpretation

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(xgb_model)

In [ ]:
dt = X_train
shap_values = explainer.shap_values(dt)

In [ ]:
sv = pd.DataFrame(shap_values)
sv.columns=["shap_"+ x for x in X_train.columns]
tr = pd.concat((X_train.reset_index(drop=True),sv), axis=1)
tr.to_csv("df_with_shap.csv",index=False)